In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/boostcamp/cascade_rcnn_swin-l-p4-w7_fpn_ms-crop-3x_coco.py')
#print(cfg.model.roi_head.bbox_head[0])
root='../dataset/'

epoch = 'latest-copy'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1024, 1024) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 8

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/cascade_rcnn_swin-l-p4-w7_fpn_ms-crop-3x_coco'

cfg.model.roi_head.bbox_head[0].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [4]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=32,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/opt/ml/detection/mmdetection/mmdet/datasets/api_wrappers/coco_api.py:22: UserWarning: mmpycocotools is deprecated. Please install official pycocotools by "pip install pycocotools"
  UserWarning)


In [5]:
# default
# cfg.model.test_cfg.rpn.nms.iou_threshold = 0.7
# cfg.model.test_cfg.rcnn.score_thr = 0.05
# cfg.model.test_cfg.rcnn.nms.iou_threshold=0.7

cfg.model.test_cfg.rpn.nms.iou_threshold = 0.7
cfg.model.test_cfg.rcnn.score_thr = 0.01
cfg.model.test_cfg.rcnn.nms.iou_threshold=0.7
print(cfg.model.test_cfg)

{'rpn': {'nms_pre': 1000, 'max_per_img': 1000, 'nms': {'type': 'nms', 'iou_threshold': 0.7}, 'min_bbox_size': 0}, 'rcnn': {'score_thr': 0.01, 'nms': {'type': 'nms', 'iou_threshold': 0.7}, 'max_per_img': 100}}


In [6]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load


/opt/ml/detection/mmdetection/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


Use load_from_local loader


In [7]:
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [8]:
output = single_gpu_test(model, data_loader, show_score_thr=0.01) # output 계산

[                                                  ] 0/4871, elapsed: 0s, ETA:

/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:324: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:361: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 5.4 task/s, elapsed: 907s, ETA:     0s

In [9]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.37337473 170.35135 415.23917 225.46725 445...,test/0000.jpg
1,0 0.016763885 336.8784 242.70834 749.723 697.5...,test/0001.jpg
2,0 0.9928808 878.8054 468.17163 1023.891 628.06...,test/0002.jpg
3,0 0.054872986 0.0 0.0 900.81055 993.0676 0 0.0...,test/0003.jpg
4,0 0.97757614 426.1117 411.59888 659.5465 571.7...,test/0004.jpg
